In [1]:
print("Hello collabski")

Hello collabski


In [2]:
import torch
import numpy as np

print(torch.__version__)
print(np.__version__)
print(torch.cuda.is_available())

2.6.0+cu124
2.0.2
True


In [6]:
import torch
import torch.nn as nn
import numpy as np

vocab_size = 6 #6 token test vocabulary
embedding_dim = 128 #vector size per token
word_embedding_layer = nn.Embedding(vocab_size, embedding_dim)

#example:
input_sentence_ids = torch.tensor([[1,3,4,5]], dtype=torch.long)

#move input to gpu
if torch.cuda.is_available():
  print("moving to gpu")
  word_embedding_layer.to('cuda') #moving the whole embed token layer
  input_sentence_ids = input_sentence_ids.to('cuda')
#pass id's through the embedding layer and then get the shape resutls
embedded_sentence = word_embedding_layer(input_sentence_ids)
print(f"sentence ids: {input_sentence_ids.shape}")
print(f"embed sentenc: {embedded_sentence.shape}")
print(f"First word's embedding:{embedded_sentence[0,0, :5]}")
print(f"is tensor in gpu? {embedded_sentence.is_cuda}")


moving to gpu
sentence ids: torch.Size([1, 4])
embed sentenc: torch.Size([1, 4, 128])
First word's embedding:tensor([-0.2959, -0.1306, -0.8948, -0.8562,  0.5636], device='cuda:0',
       grad_fn=<SliceBackward0>)
is tensor in gpu? True
